# Lamport clocks in python

In [1]:
import zmq
import pickle
context = zmq.Context() 
name="A"
other_processes=["B","C"]
urlA="tcp://"+ "127.0.0.1" +":"+ "6781"   
urlB="tcp://"+ "127.0.0.1" +":"+ "6782" 
urlC="tcp://"+ "127.0.0.1" +":"+ "6783" 
urls={"A":urlA,"B":urlB,"C":urlC}

sockets_send={}
sockets_send[name] = context.socket(zmq.PUSH)
sockets_send[name].connect(urls[name])

for i in other_processes:
    sockets_send[i] = context.socket(zmq.PUSH)
    sockets_send[i].connect(urls[i])

socket_receive = context.socket(zmq.PULL)
print("listens: ",urls[name])
socket_receive.bind(urls[name])

listens:  tcp://127.0.0.1:6781


In [2]:
def send_msg_request( p, m):
    print(" - Sent message from {} to {}".format("A",p))
    print(" sent to ",urls[p])
    sockets_send[p].send(pickle.dumps((m))) # not blocking if socket PULL is up and listening

In [3]:
from multiprocessing import Queue
queue=Queue()
contador=0
def read_msg_request(socket_receive):
    global contador
    while contador<10:
        msg=pickle.loads(socket_receive.recv()) # blocks
        if msg!="STOP":
            queue.put(msg)
            print("Thread: put message into queue: ",msg)
            send_msg_request( "B", "HOLA B")
            contador=contador+1
            print("Thread: sent message to B")
        else:
            print("Stop receiving requests")
            socket_receive.close() 
            break

In [4]:
import threading

threading.Thread(target=read_msg_request, args=(socket_receive,)).start() 

Thread: put message into queue:  HOLA A
 - Sent message from A to B
 sent to  tcp://127.0.0.1:6782
Thread: sent message to B
Thread: put message into queue:  HOLA A
 - Sent message from A to B
 sent to  tcp://127.0.0.1:6782
Thread: sent message to B
Thread: put message into queue:  HOLA A
 - Sent message from A to B
 sent to  tcp://127.0.0.1:6782
Thread: sent message to B
Thread: put message into queue:  HOLA A
 - Sent message from A to B
 sent to  tcp://127.0.0.1:6782
Thread: sent message to B
Thread: put message into queue:  HOLA A
 - Sent message from A to B
 sent to  tcp://127.0.0.1:6782
Thread: sent message to B
Thread: put message into queue:  HOLA A
 - Sent message from A to B
 sent to  tcp://127.0.0.1:6782
Thread: sent message to B
Thread: put message into queue:  HOLA A
 - Sent message from A to B
 sent to  tcp://127.0.0.1:6782
Thread: sent message to B
Thread: put message into queue:  HOLA A
 - Sent message from A to B
 sent to  tcp://127.0.0.1:6782
Thread: sent message to B
